In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


df = pd.read_csv('historical_data.csv')



df['Date'] = pd.to_datetime(df['Date'])  # Converts the Date column (which is text) into proper datetime objects, so Python understands it as dates.
df.set_index('Date', inplace=True)

print("✅ Dataset loaded successfully!")
print(df.head())


# Use only the 'Close' price for trend prediction
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Close']].values) # Takes only the “Close” prices and scales them between 0 and 1.

# Function to create training sequences
def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
        y.append(data[i])
    return np.array(X), np.array(y)

sequence_length = 60  # last 60 days → next day prediction
X, y = create_sequences(scaled_data, sequence_length)

# Split into train/test (80/20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"📊 Training samples: {len(X_train)} | Testing samples: {len(X_test)}")

# =========================
#  STEP 3 — BUILD LSTM MODEL
# =========================
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# =========================
#  STEP 4 — TRAIN MODEL
# =========================
print("🚀 Training the model...")
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test))

# =========================
#  STEP 5 — PREDICT & FIND TREND
# =========================
predicted = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test)

# Convert to trend labels
trend = np.where(predicted_prices[1:] > predicted_prices[:-1], 'Up', 'Down')

# Display sample predictions
print("\n📈 Sample Predicted Trends:")
for i in range(10):
    print(f"Day {i+1}: {trend[i]}")

# =========================
#  STEP 6 — (Optional) SAVE MODEL
# =========================
model.save('trend_model.h5')
print("\n✅ Model saved as 'trend_model.h5'")



# ['Up']	The model predicts that tomorrow’s closing price will be higher than today’s — an upward trend.
# ['Down']	The model predicts that tomorrow’s closing price will be lower than today’s — a downward trend.


✅ Dataset loaded successfully!
              Open    High     Low   Close  Volume
Date                                              
2022-01-03  101.25  102.48   98.18  100.50   48712
2022-01-04  100.82  103.05   98.19  100.36   43722
2022-01-05  101.63  102.94  100.41  101.01   20625
2022-01-06  102.87  104.43  102.33  102.55   11629
2022-01-07  102.70  102.51  100.48  102.31   20973
📊 Training samples: 577 | Testing samples: 145
🚀 Training the model...
Epoch 1/25


c:\Users\kush2\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - loss: 0.0448 - val_loss: 0.0030
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0085 - val_loss: 0.0036
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0062 - val_loss: 0.0057
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0078 - val_loss: 0.0044
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0065 - val_loss: 0.0050
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0052 - val_loss: 0.0034
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0052 - val_loss: 0.0041
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0053 - val_loss: 0.0027
Epoch 9/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0051 - val_loss: 0.0027
Epoch 10/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0045 - val_loss: 0.0030
Epoch 11/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0046 - val_loss: 0.0034
Epoch 12/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0046 - val_l


📈 Sample Predicted Trends:
Day 1: ['Down']
Day 2: ['Down']
Day 3: ['Down']
Day 4: ['Down']
Day 5: ['Down']
Day 6: ['Down']
Day 7: ['Down']
Day 8: ['Down']
Day 9: ['Down']
Day 10: ['Down']

✅ Model saved as 'trend_model.h5'
